# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression  # Adicione esta linha
from mlxtend.feature_selection import SequentialFeatureSelector


In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
# Defina a fração de dados que você deseja para o conjunto de teste (25% neste caso)
test_size = 0.25

# Use a função train_test_split para dividir os dados em treinamento e teste
train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)

# O conjunto de treinamento estará em train_df e o conjunto de teste estará em test_df

In [5]:
# Carregue os dados
df = pd.read_csv('previsao_de_renda.csv')

# Lidar com variáveis categóricas usando codificação de rótulos
label_encoder = LabelEncoder()
categorical_columns = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Converter a coluna 'data_ref' em uma representação numérica (por exemplo, dias desde uma data de referência)
df['data_ref'] = (pd.to_datetime(df['data_ref']) - pd.to_datetime('2000-01-01')).dt.days

# Separar os dados em características (X) e alvo (y)
X = df.drop(columns=['renda'])
y = df['renda']

# Tratar valores ausentes (NaN) usando imputação
imputer = SimpleImputer(strategy='mean')  # Substituir NaNs pela média dos valores existentes
X = imputer.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste (25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Definir os valores de alpha que você deseja testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Criar uma lista para armazenar os resultados de R²
r2_scores = []

# Iterar pelos diferentes valores de alpha e ajustar o modelo Ridge
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    
    # Fazer previsões no conjunto de teste
    y_pred = ridge.predict(X_test)
    
    # Calcular o R² e armazená-lo na lista
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Encontrar o valor de alpha que resultou no melhor R²
best_alpha = alphas[r2_scores.index(max(r2_scores))]
best_r2 = max(r2_scores)

print(f"Melhor valor de alpha: {best_alpha}")
print(f"Melhor R²: {best_r2}")



Melhor valor de alpha: 0
Melhor R²: 0.2626387765366982


In [6]:
# Carregue os dados
df = pd.read_csv('previsao_de_renda.csv')

# Lidar com variáveis categóricas usando codificação de rótulos
label_encoder = LabelEncoder()
categorical_columns = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Converter a coluna 'data_ref' em uma representação numérica (por exemplo, dias desde uma data de referência)
df['data_ref'] = (pd.to_datetime(df['data_ref']) - pd.to_datetime('2000-01-01')).dt.days

# Separar os dados em características (X) e alvo (y)
X = df.drop(columns=['renda'])
y = df['renda']

# Tratar valores ausentes (NaN) usando imputação
imputer = SimpleImputer(strategy='mean')  # Substituir NaNs pela média dos valores existentes
X = imputer.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste (25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Definir os valores de alpha que você deseja testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Crie uma lista para armazenar os resultados de R² para Lasso
r2_scores_lasso = []

# Iterar pelos diferentes valores de alpha e ajustar o modelo Lasso
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    
    # Fazer previsões no conjunto de teste
    y_pred = lasso.predict(X_test)
    
    # Calcular o R² e armazená-lo na lista
    r2 = r2_score(y_test, y_pred)
    r2_scores_lasso.append(r2)

# Encontrar o valor de alpha que resultou no melhor R² para Lasso
best_alpha_lasso = alphas[r2_scores_lasso.index(max(r2_scores_lasso))]
best_r2_lasso = max(r2_scores_lasso)

print(f"Melhor valor de alpha para Lasso: {best_alpha_lasso}")
print(f"Melhor R² para Lasso: {best_r2_lasso}")


/Users/samwalford/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/Users/samwalford/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/samwalford/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.884e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV

Melhor valor de alpha para Lasso: 0.1
Melhor R² para Lasso: 0.26263986198736244


/Users/samwalford/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.881e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


In [7]:
pip install mlxtend


Note: you may need to restart the kernel to use updated packages.


In [8]:
# Carregue os dados
df = pd.read_csv('previsao_de_renda.csv')

# Lidar com variáveis categóricas usando codificação de rótulos
label_encoder = LabelEncoder()
categorical_columns = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Converter a coluna 'data_ref' em uma representação numérica (por exemplo, dias desde uma data de referência)
df['data_ref'] = (pd.to_datetime(df['data_ref']) - pd.to_datetime('2000-01-01')).dt.days

# Separar os dados em características (X) e alvo (y)
X = df.drop(columns=['renda'])
y = df['renda']

# Tratar valores ausentes (NaN) usando imputação
imputer = SimpleImputer(strategy='mean')  # Substituir NaNs pela média dos valores existentes
X = imputer.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste (25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Inicializar um modelo de regressão linear
model = LinearRegression()

# Inicializar o sequencial feature selector para seleção de recursos stepwise
sfs = SequentialFeatureSelector(model, forward=True, scoring='r2', cv=5)

# Realizar seleção de recursos stepwise no conjunto de treinamento
sfs.fit(X_train, y_train)

# Obter o índice das características selecionadas
selected_features_idx = sfs.k_feature_idx_

# Selecionar as características correspondentes no conjunto de teste
X_test_selected = X_test[:, selected_features_idx]

# Treinar o modelo usando as características selecionadas
model.fit(X_train[:, selected_features_idx], y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test_selected)

# Calcular o R²
r2 = r2_score(y_test, y_pred)

print(f"R² na base de teste após seleção de recursos stepwise: {r2}")


R² na base de teste após seleção de recursos stepwise: 0.14114639596819656


In [9]:
# Carregue os dados
df = pd.read_csv('previsao_de_renda.csv')

# Lidar com variáveis categóricas usando codificação de rótulos
label_encoder = LabelEncoder()
categorical_columns = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Converter a coluna 'data_ref' em uma representação numérica (por exemplo, dias desde uma data de referência)
df['data_ref'] = (pd.to_datetime(df['data_ref']) - pd.to_datetime('2000-01-01')).dt.days

# Separar os dados em características (X) e alvo (y)
X = df.drop(columns=['renda'])
y = df['renda']

# Tratar valores ausentes (NaN) usando imputação
imputer = SimpleImputer(strategy='mean')  # Substituir NaNs pela média dos valores existentes
X = imputer.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste (25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Inicializar modelos Ridge e Lasso
ridge_model = Ridge(alpha=0.01)  # Escolha o alpha adequado
lasso_model = Lasso(alpha=0.01)  # Escolha o alpha adequado

# Treinar modelos Ridge e Lasso
ridge_model.fit(X_train, y_train)
lasso_model.fit(X_train, y_train)

# Calcular o R² para modelos Ridge e Lasso na base de teste
y_pred_ridge = ridge_model.predict(X_test)
y_pred_lasso = lasso_model.predict(X_test)

r2_ridge = r2_score(y_test, y_pred_ridge)
r2_lasso = r2_score(y_test, y_pred_lasso)

# Inicializar o sequencial feature selector para seleção de recursos stepwise
sfs = SequentialFeatureSelector(LinearRegression(), forward=True, scoring='r2', cv=5)

# Realizar seleção de recursos stepwise no conjunto de treinamento
sfs.fit(X_train, y_train)

# Obter o índice das características selecionadas
selected_features_idx = sfs.k_feature_idx_

# Selecionar as características correspondentes no conjunto de teste
X_test_selected = X_test[:, selected_features_idx]

# Treinar um modelo de regressão linear com as características selecionadas
model_stepwise = LinearRegression()
model_stepwise.fit(X_train[:, selected_features_idx], y_train)

# Fazer previsões no conjunto de teste usando o modelo stepwise
y_pred_stepwise = model_stepwise.predict(X_test_selected)

# Calcular o R² para o modelo stepwise na base de teste
r2_stepwise = r2_score(y_test, y_pred_stepwise)

# Número de recursos selecionados pelo modelo stepwise
num_selected_features = len(selected_features_idx)

# Imprimir os resultados
print(f"R² para Ridge: {r2_ridge}")
print(f"R² para Lasso: {r2_lasso}")
print(f"R² para Stepwise: {r2_stepwise}")
print(f"Número de recursos selecionados pelo modelo stepwise: {num_selected_features}")


R² para Ridge: 0.26263874793913045
R² para Lasso: 0.26263844994183194
R² para Stepwise: 0.14114639596819656
Número de recursos selecionados pelo modelo stepwise: 1


/Users/samwalford/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.884e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Qual método chega a um melhor resultado?
 
Com base nos resultados de R² na base de teste, tanto Ridge quanto Lasso parecem oferecer um desempenho semelhante e superior ao modelo de seleção de recursos stepwise. A escolha entre Ridge e Lasso depende da importância da interpretabilidade (Ridge) e da seleção automática de características (Lasso) para o seu problema específico.

In [10]:
# Carregue os dados
df = pd.read_csv('previsao_de_renda.csv')

# Lidar com variáveis categóricas usando codificação de rótulos
label_encoder = LabelEncoder()
categorical_columns = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Converter a coluna 'data_ref' em uma representação numérica (por exemplo, dias desde uma data de referência)
df['data_ref'] = (pd.to_datetime(df['data_ref']) - pd.to_datetime('2000-01-01')).dt.days

# Separar os dados em características (X) e alvo (y)
X = df.drop(columns=['renda'])
y = df['renda']

# Tratar valores ausentes (NaN) usando imputação
imputer = SimpleImputer(strategy='mean')  # Substituir NaNs pela média dos valores existentes
X = imputer.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste (25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Pipeline para aplicar transformação polinomial e treinar o modelo Ridge
polynomial_pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),  # Grau 2 para características polinomiais
    ('ridge', Ridge(alpha=0.01))  # Ajuste o valor de alpha conforme necessário
])

# Treinar o modelo no conjunto de treinamento
polynomial_pipe.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = polynomial_pipe.predict(X_test)

# Calcular o R²
r2 = r2_score(y_test, y_pred)

print(f"R² com transformação polinomial: {r2}")


R² com transformação polinomial: 0.3757186172897421


/Users/samwalford/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=7.36541e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [11]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [12]:
# Carregue os dados
df = pd.read_csv('previsao_de_renda.csv')

# Lidar com variáveis categóricas usando codificação de rótulos
label_encoder = LabelEncoder()
categorical_columns = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Converter a coluna 'data_ref' em uma representação numérica (por exemplo, dias desde uma data de referência)
df['data_ref'] = (pd.to_datetime(df['data_ref']) - pd.to_datetime('2000-01-01')).dt.days

# Separar os dados em características (X) e alvo (y)
X = df.drop(columns=['renda'])
y = df['renda']

# Tratar valores ausentes (NaN) usando imputação
imputer = SimpleImputer(strategy='mean')  # Substituir NaNs pela média dos valores existentes
X = imputer.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste (25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Inicializar um modelo de árvore de regressão
tree_model = DecisionTreeRegressor(random_state=42)  # Você pode ajustar hiperparâmetros conforme necessário

# Treinar o modelo
tree_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = tree_model.predict(X_test)

# Calcular o R²
r2 = r2_score(y_test, y_pred)

print(f"R² com árvore de regressão: {r2}")


R² com árvore de regressão: 0.15891218627802595
